<a href="https://colab.research.google.com/github/jonlwowski012/StockSellRulesChecker/blob/master/StockScreener/Screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/jonlwowski012/StockSellRulesChecker.git
! pip install -r StockSellRulesChecker/StockScreener/requirements.txt

Cloning into 'StockSellRulesChecker'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 60 (delta 18), reused 26 (delta 6), pack-reused 0
Unpacking objects: 100% (60/60), done.
     |████████████████████████████████| 9.5MB 8.7MB/s 
     |████████████████████████████████| 1.3MB 40.7MB/s 
     |████████████████████████████████| 143kB 38.7MB/s 
     |████████████████████████████████| 296kB 45.8MB/s 
  Created wheel for finviz: filename=finviz-1.3.4-cp37-none-any.whl size=15193 sha256=db13ca05a86b0a81d4781c032ee44c82f74feca8753f6893d8f2f528c45147f1
  Stored in directory: /root/.cache/pip/wheels/6d/fa/7c/4cc075d8af3f8ed083dd112b1a2b09888b9a9a2964c907291f
  Created wheel for stockquotes: filename=stockquotes-2.0.1-cp37-none-any.whl size=4472 sha256=a52973f46729d162df4a46ba5fa5eb124db9efcfe35eed525e596c09ced4b97a
  Stored in directory: /root/.cache/pip/wheels/b6/5a/fa/c250a28690344108071fb9b467

In [10]:
from finviz.screener import Screener
import nest_asyncio
import finviz
import pandas_datareader as pdr
import yfinance as yf
import datetime
import stockquotes
import pandas as pd
import tqdm
nest_asyncio.apply()

In [39]:
eps_5year_over5pct = True
eps_qoq_over5pct = True
eps_yoy_over5pct = True
sales_5years_over5pct = True
sales_qoq_over5pct = True

filters = []
if eps_5year_over5pct:
  filters.append('fa_eps5years_o5')
if eps_qoq_over5pct:
  filters.append('fa_epsqoq_o5')
if eps_yoy_over5pct:
  filters.append('fa_epsyoy_o5')
if sales_5years_over5pct:
  filters.append('fa_sales5years_o5')
if sales_qoq_over5pct:
  filters.append('fa_salesqoq_o5')

stock_list = Screener(filters=filters, table='Performance', order='price')  

In [40]:
print(stock_list)

No. | Ticker | Perf Week | Perf Month | Perf Quart | Perf Half | Perf Year | Perf YTD | Volatility W | Volatility M | Recom | Avg Volume | Rel Volume | Price  | Change | Volume   
--- | ------ | --------- | ---------- | ---------- | --------- | --------- | -------- | ------------ | ------------ | ----- | ---------- | ---------- | ------ | ------ | ---------
1   | ISR    | 30.36%    | -17.51%    | 255.06%    | 116.26%   | 160.57%   | 224.44%  | 11.96%       | 12.96%       | 2.00  | 16.23M     | 0.22       | 1.38   | -5.14% | 2,757,898
2   | CHNR   | 9.24%     | -12.61%    | 40.56%     | 82.73%    | 102.01%   | 45.65%   | 8.28%        | 12.75%       | -     | 1.88M      | 0.27       | 2.05   | 1.99%  | 391,355  
3   | NBRV   | 17.14%    | -35.13%    | -17.67%    | -64.06%   | -65.94%   | -15.29%  | 12.93%       | 10.67%       | 2.70  | 2.09M      | 1.69       | 2.06   | 0.24%  | 2,737,158
4   | PRPO   | 11.21%    | -30.65%    | 15.18%     | 18.89%    | 165.98%   | 24.64%   | 8.43%       

In [41]:
def moving_average(yahoo_df, days):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max()
    end_date = curr_date - datetime.timedelta(days=days)
    
    df_days = df[df['Date'] >= end_date]
    return float(df_days['Close'].mean())

def week52_low_high(yahoo_df):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max()
    end_date = curr_date - datetime.timedelta(days=365)
    
    df_days = df[df['Date'] >= end_date]
    return float(df_days['Close'].max()), float(df_days['Close'].min())


screened_stocks = {}
for stock in tqdm.tqdm(stock_list):
  screened_stocks[stock['Ticker']] = {}
  
  print(stock['Ticker'])
  finviz_stats = finviz.get_stock(stock['Ticker'])
  try:
      yahoo_df = pdr.get_data_yahoo(stock['Ticker'], interval = "d")
  except:
      continue

  SMA200_value = moving_average(yahoo_df, days=200)
  SMA150_value = moving_average(yahoo_df, days=150)
  SMA50_value = moving_average(yahoo_df, days=50)
  SMA200_percent = float(finviz_stats['SMA200'].replace("%",""))
  SMA50_percent = float(finviz_stats['SMA50'].replace("%",""))

  try:
    EPS_ttm = float(finviz_stats['EPS (ttm)'])
  except:
    EPS_ttm = 0

  try:
    EPS_QoQ_percent = float(finviz_stats['EPS Q/Q'].replace("%",""))
  except:
    EPS_QoQ_percent = 0

  try:
    Sales_QoQ_percent = float(finviz_stats['Sales Q/Q'].replace("%",""))
  except:
    Sales_QoQ_percent = 0

  try:
    inst_own = float(finviz_stats['Inst Own'].replace("%",""))/100
  except:
    inst_own = 0
  
  shares_outstanding = finviz_stats['Shs Outstand']
  if 'M' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("M", ""))*1e6
  elif 'B' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("B", ""))*1e9
  elif 'T' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("T", ""))*1e12

  prev_close = float(finviz_stats['Prev Close'].replace("$",""))
  week52_high, week52_low = week52_low_high(yahoo_df)
  
  screened_stocks[stock['Ticker']]['SMA200_value'] = SMA200_value
  screened_stocks[stock['Ticker']]['SMA150_value'] = SMA150_value
  screened_stocks[stock['Ticker']]['SMA50_value'] = SMA50_value
  screened_stocks[stock['Ticker']]['SMA200_percent'] = SMA200_percent
  screened_stocks[stock['Ticker']]['SMA50_percent'] = SMA50_percent
  screened_stocks[stock['Ticker']]['EPS_QoQ_percent'] = EPS_QoQ_percent
  screened_stocks[stock['Ticker']]['EPS_ttm'] = EPS_ttm
  screened_stocks[stock['Ticker']]['Sales_QoQ_percent'] = Sales_QoQ_percent
  screened_stocks[stock['Ticker']]['prev_close'] = prev_close
  screened_stocks[stock['Ticker']]['week52_high'] = week52_high
  screened_stocks[stock['Ticker']]['week52_low'] = week52_low
  screened_stocks[stock['Ticker']]['Inst. Ownership'] = inst_own
  screened_stocks[stock['Ticker']]['Shares Outstanding'] = shares_outstanding
  
  # EPS TTM > $1 Rule
  if EPS_ttm >= 1.0:
    eps_ttm_rule = True
  else:
    eps_ttm_rule = False

  # EPS QoQ Yearly > 18% rule
  if EPS_QoQ_percent >= 0.18:
    eps_QoQ_yearly_rule = True
  else:
    eps_QoQ_yearly_rule = False
  screened_stocks[stock['Ticker']]['eps_QoQ_yearly_rule'] = eps_QoQ_yearly_rule

  # Sales QoQ Yearly > 25% rule
  if Sales_QoQ_percent >= 0.25:
    sales_QoQ_yearly_rule = True
  else:
    sales_QoQ_yearly_rule = False
  screened_stocks[stock['Ticker']]['sales_QoQ_yearly_rule'] = sales_QoQ_yearly_rule

  # Shares Outstanding <= 25 mil
  if shares_outstanding <= 25e6:
    shares_outstanding_rule = True
  else:
    shares_outstanding_rule = False
  screened_stocks[stock['Ticker']]['shares_outstanding_rule'] = shares_outstanding_rule

  # Institutional Ownership < 35%
  if 0.05 <= inst_own <= 0.35:
    inst_ownership_rule = True
  else:
    inst_ownership_rule = False
  screened_stocks[stock['Ticker']]['inst_ownership_rule'] = inst_ownership_rule

  # Positive 200d MA positive
  if SMA200_percent > 0:
      SMA200_slope_rule = True
  else:
      SMA200_slope_rule = False
  screened_stocks[stock['Ticker']]['SMA200_slope_rule'] = SMA200_slope_rule
      
  # 150d MA greater than 200d MA
  if SMA150_value > SMA200_value:
      SMA150_greater_SMA200_rule = True
  else:
      SMA150_greater_SMA200_rule = False
  screened_stocks[stock['Ticker']]['SMA150_greater_SMA200_rule'] = SMA150_greater_SMA200_rule
      
  # 50d MA greater than 150d MA
  if SMA50_value > SMA150_value:
      SMA50_greater_SMA150_rule = True
  else:
      SMA50_greater_SMA150_rule = False
  screened_stocks[stock['Ticker']]['SMA50_greater_SMA150_rule'] = SMA50_greater_SMA150_rule
      
  # Close above 50d MA
  if prev_close > SMA50_value:
      close_greater_SMA50_rule = True
  else:
      close_greater_SMA50_rule = False
  screened_stocks[stock['Ticker']]['close_greater_SMA50_rule'] = close_greater_SMA50_rule
      
  # 52 week high low span rule
  if 0.75*week52_high > 1.25*week52_low:
      week52_span_rule = True
  else:
      week52_span_rule = False
  screened_stocks[stock['Ticker']]['week52_span_rule'] = week52_span_rule
  
  # Close above 52 week high - 25%
  if prev_close > 0.75*week52_high:
      close_above_52weekhigh_rule = True
  else:
      close_above_52weekhigh_rule = False
  screened_stocks[stock['Ticker']]['close_above_52weekhigh_rule'] = close_above_52weekhigh_rule

output_list = []
for stock in screened_stocks.keys():
    cols = ["Ticker"] + list(screened_stocks[stock].keys())
    temp_list = []
    temp_list.append(stock)
    for rule in screened_stocks[stock].keys():
        temp_list.append(screened_stocks[stock][rule])
    output_list.append(temp_list)
        
df_out = pd.DataFrame(output_list,columns=cols)

    



  0%|          | 0/356 [00:00<?, ?it/s]

ISR



  0%|          | 1/356 [00:00<04:55,  1.20it/s]

CHNR



  1%|          | 2/356 [00:01<04:48,  1.23it/s]

...


 28%|██▊       | 98/356 [02:55<11:33,  2.69s/it]

EXR



 28%|██▊       | 99/356 [02:57<10:54,  2.55s/it]

KMX



 28%|██▊       | 100/356 [03:03<07:49,  1.83s/it]


In [42]:
rules = [col for col in df_out.columns if "rule" in col]
df_out['Passed Tests'] = (df_out[list(rules)]).sum(1)

In [43]:
df_out.sort_values('Passed Tests', ascending=False)

,Ticker,SMA200_value,SMA150_value,SMA50_value,SMA200_percent,SMA50_percent,EPS_QoQ_percent,EPS_ttm,Sales_QoQ_percent,prev_close,week52_high,week52_low,Inst. Ownership,Shares Outstanding,eps_QoQ_yearly_rule,sales_QoQ_yearly_rule,shares_outstanding_rule,inst_ownership_rule,SMA200_slope_rule,SMA150_greater_SMA200_rule,SMA50_greater_SMA150_rule,close_greater_SMA50_rule,week52_span_rule,close_above_52weekhigh_rule,Passed Tests
25,ESCA,20.570942,21.167019,21.823611,20.76,4.00,98.5,1.82,59.1,22.91,24.049999,4.750000,0.307,14030000.0,True,True,True,True,True,True,True,True,True,True,10
10,MOSY,2.498696,2.803365,4.387778,92.46,11.34,76.0,-1.44,66.7,4.71,6.260000,0.840000,0.096,3550000.0,True,True,True,True,True,True,True,True,True,True,10
26,STXB,16.353442,17.746106,20.912778,54.78,16.01,108.6,1.79,24.4,22.57,24.180000,9.020000,0.389,17170000.0,True,True,True,False,True,True,True,True,True,True,9
57,AB,33.012391,34.510961,37.775556,31.88,11.27,10.1,2.84,15.4,40.37,41.220001,14.660000,0.187,101230000.0,True,True,False,True,True,True,True,True,True,True,9
31,CODI,20.028913,20.692019,22.932222,26.02,7.62,119.8,-0.21,22.7,23.77,24.430000,11.150000,0.334,64940000.0,True,True,False,True,True,True,True,True,True,True,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,EBS,98.319167,96.462837,104.394584,-7.95,-13.47,283.0,5.63,61.8,91.65,134.940002,49.340000,0.849,53300000.0,True,True,False,False,False,False,True,False,True,False,4
40,PINC,34.201848,34.768317,34.264028,-0.34,-2.38,105.2,4.23,32.3,34.00,37.419998,27.400000,0.675,122130000.0,True,True,False,False,False,True,False,False,False,True,4
52,WPM,43.960326,41.719087,38.912917,-11.79,-0.27,101.8,1.13,28.2,40.12,56.209999,23.740000,0.641,449320000.0,True,True,False,False,False,False,False,True,True,False,4
9,QTNT,5.449348,5.523077,4.815000,-31.36,-19.46,20.9,-1.10,11.4,4.13,9.260000,2.540000,0.923,101020000.0,True,True,False,False,False,True,False,False,True,False,4
